In [1]:
import os
os.chdir(r"C:\Users\KutayTaha\Desktop\diş_eti")
print(os.getcwd())


c:\Users\KutayTaha\Desktop\diş_eti


# ilk

In [2]:
import os
from PIL import Image
from collections import defaultdict


ModuleNotFoundError: No module named 'PIL'

In [3]:
BASE_DIR = "Dataset"   # proje kökünden Dataset klasörü
SPLITS = ["training", "validation", "test"]


In [4]:
def check_dataset(split):
    img_dir = os.path.join(BASE_DIR, split, "images")
    lbl_dir = os.path.join(BASE_DIR, split, "labels")

    images = {os.path.splitext(f)[0] for f in os.listdir(img_dir)}
    labels = {os.path.splitext(f)[0] for f in os.listdir(lbl_dir)}

    missing_labels = images - labels      # resmi var label yok
    missing_images = labels - images      # label var resim yok

    corrupted_images = []
    for img in os.listdir(img_dir):
        img_path = os.path.join(img_dir, img)
        try:
            Image.open(img_path).verify()
        except Exception:
            corrupted_images.append(img)

    return missing_labels, missing_images, corrupted_images


In [5]:
results = {}

for split in SPLITS:
    results[split] = check_dataset(split)

results



{'training': (set(), set(), []),
 'validation': (set(), set(), []),
 'test': (set(), set(), [])}

In [6]:
def check_label_format(label_path):
    errors = []

    with open(label_path, "r") as f:
        for i, line in enumerate(f):
            parts = line.strip().split()

            if len(parts) != 5:
                errors.append(f"Satır {i}: 5 eleman yok")
                continue

            try:
                cls = int(parts[0])
                coords = list(map(float, parts[1:]))

                if not all(0 <= c <= 1 for c in coords):
                    errors.append(f"Satır {i}: Koordinatlar 0-1 dışında")
            except:
                errors.append(f"Satır {i}: Sayısal dönüşüm hatası")

    return errors



In [7]:
label_errors = defaultdict(list)

for split in SPLITS:
    lbl_dir = os.path.join(BASE_DIR, split, "labels")

    for lbl in os.listdir(lbl_dir):
        path = os.path.join(lbl_dir, lbl)
        errs = check_label_format(path)

        if errs:
            label_errors[split].append((lbl, errs))

label_errors


defaultdict(list, {'training': [('00598.txt', ['Satır 14: 5 eleman yok'])]})

In [10]:
label_path = "Dataset/training/labels/00598.txt"

with open(label_path, "r") as f:
    for i, line in enumerate(f, start=1):
        if i == 14:
            print(f"Satır {i}: '{line.strip()}'")



Satır 14: '0 0.541304 0.520257 0.584239 0.289699'


In [11]:
def clean_all_labels():
    for split in ["training", "validation", "test"]:
        lbl_dir = f"Dataset/{split}/labels"

        for lbl in os.listdir(lbl_dir):
            path = os.path.join(lbl_dir, lbl)

            with open(path, "r") as f:
                lines = f.readlines()

            cleaned = [
                line for line in lines
                if len(line.strip().split()) == 5
            ]

            if len(cleaned) != len(lines):
                with open(path, "w") as f:
                    f.writelines(cleaned)
                print(f"Düzeltildi: {split}/{lbl}")

clean_all_labels()


Düzeltildi: training/00598.txt


In [12]:
from collections import Counter

class_counter = Counter()

for split in SPLITS:
    lbl_dir = os.path.join(BASE_DIR, split, "labels")

    for lbl in os.listdir(lbl_dir):
        with open(os.path.join(lbl_dir, lbl)) as f:
            for line in f:
                cls = int(line.split()[0])
                class_counter[cls] += 1

class_counter


Counter({5: 4867, 4: 3851, 6: 2292, 3: 1831, 0: 1096, 1: 1096, 2: 309})

# class inbalance düzeltme

In [16]:
import os
import shutil
import random
from collections import Counter

LABEL_DIR = "Dataset/Training/labels"
IMAGE_DIR = "Dataset/Training/images"

TARGET_COUNT = 1000  # az sınıflar için hedef

# sınıf sayıları
class_counts = Counter()

label_files = os.listdir(LABEL_DIR)

for lf in label_files:
    with open(os.path.join(LABEL_DIR, lf)) as f:
        for line in f:
            cls = int(line.split()[0])
            class_counts[cls] += 1

print("Mevcut dağılım:", class_counts)

# az olan sınıflar
minor_classes = [c for c, n in class_counts.items() if n < TARGET_COUNT]

print("Çoğaltılacak sınıflar:", minor_classes)

for cls in minor_classes:
    needed = TARGET_COUNT - class_counts[cls]

    cls_files = []
    for lf in label_files:
        with open(os.path.join(LABEL_DIR, lf)) as f:
            for line in f:
                if int(line.split()[0]) == cls:
                    cls_files.append(lf)
                    break

    for i in range(needed):
        src_label = random.choice(cls_files)
        src_img = src_label.replace(".txt", ".jpg")

        new_label = f"aug_{cls}_{i}_{src_label}"
        new_img = new_label.replace(".txt", ".jpg")

        shutil.copy(
            os.path.join(LABEL_DIR, src_label),
            os.path.join(LABEL_DIR, new_label)
        )
        shutil.copy(
            os.path.join(IMAGE_DIR, src_img),
            os.path.join(IMAGE_DIR, new_img)
        )

print("Oversampling tamamlandı.")


Mevcut dağılım: Counter({5: 3399, 4: 2178, 6: 1751, 3: 1218, 0: 732, 1: 732, 2: 244})
Çoğaltılacak sınıflar: [0, 1, 2]
Oversampling tamamlandı.


In [16]:
import os
from collections import Counter

LABEL_DIR = "Dataset/Training/labels"  # main dosyandaki label yolu

class_counts = Counter()

for label_file in os.listdir(LABEL_DIR):
    if not label_file.endswith(".txt"):
        continue

    with open(os.path.join(LABEL_DIR, label_file), "r") as f:
        for line in f:
            if line.strip() == "":
                continue
            class_id = int(line.split()[0])
            class_counts[class_id] += 1

print("📊 Train sınıf dağılımı:")
for cls, count in sorted(class_counts.items()):
    print(f"Class {cls}: {count}")



📊 Train sınıf dağılımı:
Class 0: 2024
Class 1: 2024
Class 2: 2235
Class 3: 6192
Class 4: 6037
Class 5: 6625
Class 6: 3222


# eğitim

In [12]:
from ultralytics import YOLO
import torch, gc, os

torch.cuda.empty_cache()
gc.collect()

print("YOLO başlatılıyor...")
model = YOLO("yolov8m.pt")

print("TRAIN başlıyor...")
model.train(
    data="dataset.yaml",
    imgsz=640,
    epochs=1,
    batch=2,
    device=0,
    project="a",
    name="TEST_KLASOR",
    exist_ok=True
)

print("BİTTİ")
print("Klasörler:", os.listdir("runs/detect"))


YOLO başlatılıyor...
TRAIN başlıyor...
Ultralytics 8.4.12  Python-3.11.9 torch-2.1.2+cu118 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=2, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=1, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=TEST_KLASOR, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mas

In [13]:
from ultralytics import YOLO
import torch, gc

torch.cuda.empty_cache()
gc.collect()

model = YOLO("yolov8m.pt")

model.train(
    data="dataset.yaml",
    imgsz=768,
    epochs=5,
    batch=4,
    optimizer="AdamW",
    lr0=0.001,
    device=0,
    amp=True,
    project="a",
    name="dis_eti_stage1",
    exist_ok=True
)


Ultralytics 8.4.12  Python-3.11.9 torch-2.1.2+cu118 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=5, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=768, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=dis_eti_stage1, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=100, perspective=

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000016F14A1B3D0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047

In [18]:
from ultralytics import YOLO
import torch, gc

torch.cuda.empty_cache()
gc.collect()

model = YOLO("runs/detect/a/dis_eti_stage1/weights/last.pt")

model.train(
    data="dataset.yaml",
    imgsz=768,
    epochs=5,          # BURASI: +5 epoch
    batch=4,
    optimizer="AdamW",
    lr0=0.001,
    device=0,
    amp=True,
    project="a",
    name="dis_eti_stage2",
    exist_ok=True
)


Ultralytics 8.4.12  Python-3.11.9 torch-2.1.2+cu118 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=5, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=768, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs/detect/a/dis_eti_stage1/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=dis_eti_stage2, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mas

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000016F3E22CE90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047

In [20]:
from ultralytics import YOLO
import torch, gc

torch.cuda.empty_cache()
gc.collect()

model = YOLO("runs/detect/a/dis_eti_stage2/weights/last.pt")

model.train(
    data="dataset.yaml",
    imgsz=768,
    epochs=10,          
    batch=4,
    optimizer="AdamW",
    lr0=0.001,
    device=0,
    amp=True,
    project="a",
    name="dis_eti_stage3",
    exist_ok=True
)


Ultralytics 8.4.12  Python-3.11.9 torch-2.1.2+cu118 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=768, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs/detect/a/dis_eti_stage2/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=dis_eti_stage3, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_ma

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000016EFAB90350>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047

In [2]:
from ultralytics import YOLO
import torch, gc

torch.cuda.empty_cache()
gc.collect()

model = YOLO("runs/detect/a/dis_eti_stage3/weights/last.pt")

model.train(
    data="dataset.yaml",
    imgsz=768,
    epochs=20,          
    batch=4,
    optimizer="AdamW",
    lr0=0.001,
    device=0,
    amp=True,
    project="a",
    name="dis_eti_stage4",
    exist_ok=True
)


Ultralytics 8.4.12  Python-3.11.9 torch-2.1.2+cu118 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=768, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs/detect/a/dis_eti_stage3/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=dis_eti_stage4, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_ma

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000017EF5EDAF10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047

In [3]:
from ultralytics import YOLO
import torch, gc

torch.cuda.empty_cache()
gc.collect()

model = YOLO("runs/detect/a/dis_eti_stage4/weights/last.pt")

model.train(
    data="dataset.yaml",
    imgsz=768,
    epochs=60,          
    batch=4,
    optimizer="AdamW",
    lr0=0.001,
    device=0,
    amp=True,
    project="a",
    name="dis_eti_stage5",
    exist_ok=True
)


Ultralytics 8.4.12  Python-3.11.9 torch-2.1.2+cu118 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=60, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=768, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs/detect/a/dis_eti_stage4/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=dis_eti_stage5, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_ma

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000017FA3DA8790>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047

In [2]:
from ultralytics import YOLO
import torch, gc

torch.cuda.empty_cache()
gc.collect()

model = YOLO("runs/detect/a/dis_eti_stage5/weights/best.pt")

model.train(
    data="dataset.yaml",
    imgsz=768,
    epochs=30,
    batch=4,
    optimizer="AdamW",
    lr0=0.0005,
    patience=15,
    cos_lr=True,
    device=0,
    amp=True,
    project="a",
    name="dis_eti_clean",
    exist_ok=True
)


Ultralytics 8.4.12  Python-3.11.9 torch-2.1.2+cu118 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=768, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs/detect/a/dis_eti_stage5/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=dis_eti_clean, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mas

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000022B7C327410>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047

# Predict

In [ ]:
from ultralytics import YOLO

model = YOLO("runs/detect/a/dis_eti_clean/weights/best.pt")

model.predict(
    source="Dataset/test/images/00929.jpg",   
    imgsz=768,
    conf=0.5,
    save=True
)



image 1/1 c:\Users\KutayTaha\Desktop\di_eti\Dataset\test\images\00929.jpg: 544x768 1 saglikli, 1 hafif_gingivitis, 1 periodontitis, 2 plaks, 4 tartars, 1 kanama, 96.5ms
Speed: 3.7ms preprocess, 96.5ms inference, 61.6ms postprocess per image at shape (1, 3, 544, 768)
Results saved to C:\Users\KutayTaha\Desktop\di_eti\runs\detect\predict6


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'saglikli', 1: 'hafif_gingivitis', 2: 'ileri_gingivitis', 3: 'periodontitis', 4: 'plak', 5: 'tartar', 6: 'kanama'}
 obb: None
 orig_img: array([[[52, 64, 92],
         [54, 66, 94],
         [56, 68, 96],
         ...,
         [48, 61, 93],
         [42, 55, 87],
         [36, 49, 81]],
 
        [[50, 62, 90],
         [50, 62, 90],
         [54, 66, 94],
         ...,
         [47, 60, 92],
         [45, 58, 90],
         [40, 53, 85]],
 
        [[53, 65, 93],
         [50, 62, 90],
         [54, 66, 94],
         ...,
         [45, 58, 90],
         [47, 60, 92],
         [47, 60, 92]],
 
        ...,
 
        [[25, 36, 56],
         [24, 35, 55],
         [27, 36, 56],
         ...,
         [42, 54, 82],
         [40, 52, 80],
         [37, 49, 77]],
 
        [[27, 38, 58],
         [25, 36, 56],
         [27, 36, 56],
        

In [7]:
results = model.predict("Dataset/test/images/00929.jpg", conf=0.5)

img = results[0].plot(
    labels=True,
    conf=False,     # score kapalı
    line_width=3    # daha kalın kutu
)

import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
plt.imshow(img)
plt.axis("off")
plt.show()



image 1/1 c:\Users\KutayTaha\Desktop\di_eti\Dataset\test\images\00929.jpg: 544x768 1 saglikli, 1 hafif_gingivitis, 1 periodontitis, 2 plaks, 4 tartars, 1 kanama, 277.3ms
Speed: 17.2ms preprocess, 277.3ms inference, 1.5ms postprocess per image at shape (1, 3, 544, 768)


<Figure size 1000x600 with 1 Axes>

# modeli tekrar kullanmak için 

In [8]:
from ultralytics import YOLO

model = YOLO("C:/Users/KutayTaha/Desktop/diş_eti/models/best.pt")



In [9]:

model.predict(
    source="Dataset/test/images/00929.jpg",
    conf=0.5,   # 0.25 yerine 0.5
    save=True
)



image 1/1 c:\Users\KutayTaha\Desktop\di_eti\Dataset\test\images\00929.jpg: 448x640 1 saglikli, 1 hafif_gingivitis, 76.9ms
Speed: 7.7ms preprocess, 76.9ms inference, 13.8ms postprocess per image at shape (1, 3, 448, 640)
Results saved to C:\Users\KutayTaha\Desktop\di_eti\runs\detect\predict5


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'saglikli', 1: 'hafif_gingivitis', 2: 'ileri_gingivitis', 3: 'periodontitis', 4: 'plak', 5: 'tartar', 6: 'kanama'}
 obb: None
 orig_img: array([[[52, 64, 92],
         [54, 66, 94],
         [56, 68, 96],
         ...,
         [48, 61, 93],
         [42, 55, 87],
         [36, 49, 81]],
 
        [[50, 62, 90],
         [50, 62, 90],
         [54, 66, 94],
         ...,
         [47, 60, 92],
         [45, 58, 90],
         [40, 53, 85]],
 
        [[53, 65, 93],
         [50, 62, 90],
         [54, 66, 94],
         ...,
         [45, 58, 90],
         [47, 60, 92],
         [47, 60, 92]],
 
        ...,
 
        [[25, 36, 56],
         [24, 35, 55],
         [27, 36, 56],
         ...,
         [42, 54, 82],
         [40, 52, 80],
         [37, 49, 77]],
 
        [[27, 38, 58],
         [25, 36, 56],
         [27, 36, 56],
        

# model kontrolü 



yolo task=detect mode=val model="C:\Users\KutayTaha\Desktop\diş_eti\runs\detect\dis_eti_final27\weights\best.pt" data=dataset.yaml


                   all        182       2548       0.49      0.688      0.544      0.374
              saglikli        182        182      0.996          1      0.995      0.847
      hafif_gingivitis        182        182      0.995          1      0.995      0.784
      ileri_gingivitis        182         30     0.0988      0.433      0.177      0.108
         periodontitis        182        306      0.205      0.202      0.199      0.108
                  plak        182        774      0.482       0.54      0.479       0.26
                tartar        182        754      0.382      0.798      0.423       0.23
                kanama        182        320      0.274      0.844      0.538      0.283
                tartar        182        754      0.382      0.798      0.423       0.23
                tartar        182        754      0.382      0.798      0.423       0.23
                kanama        182        320      0.274      0.844      0.538      0.283

### ÇOK İYİ OLANLAR

saglikli
hafif_gingivitis


### ORTA – KABUL EDİLEBİLİR

plak
tartar
kanama


### ZAYIF OLANLAR

ileri_gingivitis
periodontitis


## Özellikle zayıf sınıflar:

ileri_gingivitis → mAP50: 0.177

periodontitis → mAP50: 0.199

tartar → mAP50: 0.423

Ama şuna dikkat:

saglikli ve hafif_gingivitis neredeyse kusursuz

Demek ki model öğrenebiliyor, sorun:

sınıf dengesizliği

zor sınıflar (görsel olarak karışık)

Yani bu kötü model değil, zor veri problemi.